In [1]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
train.shape

(891, 12)

In [10]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train = train.drop('PassengerId', axis=1)
test_ids = test.pop('PassengerId')

In [4]:
testF = train.pop('Survived')
combinedSet = pd.concat([train,test], axis = 0, ignore_index = True)
categoricalCols = combinedSet.select_dtypes(include=['object']).columns.tolist()
numericalCols = combinedSet.select_dtypes(include=['number']).columns.tolist()
OTS = len(train)
trainF = combinedSet.iloc[:OTS]
test = combinedSet.iloc[OTS:]

In [5]:
numerical_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(random_state=0))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value',
 unknown_value=-1)) 
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numericalCols),
        ('cat', categorical_transformer, categoricalCols)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', RandomForestClassifier(random_state = 42, max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=500))])

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(trainF, testF , test_size=0.2)

In [7]:
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_valid)
accuracy = accuracy_score(y_valid, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8603351955307262


In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__n_estimators': [100, 200, 500], 
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train) 

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

best_rf_model = grid_search.best_estimator_

Best parameters found:  {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Best cross-validation score:  0.8062346104599625


In [9]:
submissionAns = pd.DataFrame(pipeline.predict(test), columns=['Survived'])
submission = pd.concat([test_ids,submissionAns], axis = 1, ignore_index = False)
submission.to_csv('submission.csv', index = False)